In [1]:
# === DELFIN Dashboard ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import py3Dmol
import os
import subprocess
import shutil
import re
import sys
import numpy as np
from pathlib import Path

# RDKit für SMILES-Konvertierung
from rdkit import Chem
from rdkit.Chem import AllChem

# stk für Metallkomplexe (optional)
try:
    import stk
    HAS_STK = True
except ImportError:
    HAS_STK = False

# Ensure local DELFIN is importable in notebooks
_deflin_root = None
for _base in [Path.cwd(), *Path.cwd().parents]:
    if (_base / 'software' / 'delfin' / 'delfin').is_dir():
        _deflin_root = _base / 'software' / 'delfin'
        break
    if (_base / 'delfin').is_dir():
        _deflin_root = _base
        break
if _deflin_root and str(_deflin_root) not in sys.path:
    sys.path.insert(0, str(_deflin_root))

# Force reload to pick up any code changes
import importlib
import delfin.config
import delfin.common.control_validator
importlib.reload(delfin.common.control_validator)
importlib.reload(delfin.config)
from delfin.config import validate_control_text

# Pfade dynamisch ermitteln
HOME = Path.home()

# Notebook-Verzeichnis ermitteln (wo dieses Notebook liegt)
try:
    import IPython
    NOTEBOOK_DIR = Path(IPython.extract_module_locals()[1]['__vsc_ipynb_file__']).parent
except:
    NOTEBOOK_DIR = Path.cwd()

def find_root_dir(start_dir: Path):
    cur = start_dir
    while True:
        if (cur / "software" / "delfin").is_dir():
            return cur
        if cur == cur.parent:
            return None
        cur = cur.parent

ROOT_DIR = find_root_dir(NOTEBOOK_DIR)
if ROOT_DIR is None:
    ROOT_DIR = HOME

BASE_DIR = ROOT_DIR / "calc"

_submit_candidates = [
    ROOT_DIR / "software" / "delfin" / "examples" / "example_Job_Submission_Scripts" / "BwUniCluster" / "submit_sh",
    NOTEBOOK_DIR / "submit_sh",
]
SUBMIT_TEMPLATES_DIR = None
for _path in _submit_candidates:
    if _path.exists():
        SUBMIT_TEMPLATES_DIR = _path
        break
if SUBMIT_TEMPLATES_DIR is None:
    SUBMIT_TEMPLATES_DIR = _submit_candidates[0]

_goat_candidates = [
    ROOT_DIR / "software" / "CONTROL_Templates" / "only_GOAT.txt",
    ROOT_DIR / "CONTROL_Templates" / "only_GOAT.txt",
    NOTEBOOK_DIR / "CONTROL_Templates" / "only_GOAT.txt",
]
ONLY_GOAT_TEMPLATE_PATH = None
for _path in _goat_candidates:
    if _path.exists():
        ONLY_GOAT_TEMPLATE_PATH = _path
        break
if ONLY_GOAT_TEMPLATE_PATH is None:
    ONLY_GOAT_TEMPLATE_PATH = _goat_candidates[0]

JOB_TYPE_TEMPLATES = {
    'Short (24h)': 'submit_short.sh',
    'Standard (48h)': 'submit_standard.sh',
    'Long (72h)': 'submit_long.sh',
    'Extra Long (120h)': 'submit_extralong.sh'
}

# === SMILES Konvertierungsfunktionen ===

def contains_metal(smiles):
    """Prüfe ob SMILES ein Metall enthält."""
    metals = ['Li', 'Na', 'K', 'Rb', 'Cs', 'Be', 'Mg', 'Ca', 'Sr', 'Ba',
              'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
              'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
              'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
              'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os',
              'Ir', 'Pt', 'Au', 'Hg', 'Al', 'Ga', 'In', 'Tl', 'Sn', 'Pb',
              'Bi', 'Po', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu']
    for metal in metals:
        if re.search(rf'\[{metal}[+\-\d\]@]', smiles, re.IGNORECASE):
            return True
        if re.search(rf'\[{metal}\]', smiles, re.IGNORECASE):
            return True
    return False




def mol_from_smiles_rdkit(smiles, allow_metal=False):
    """Robuste RDKit-SMILES-Erzeugung, toleranter bei Metallkomplexen."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return mol, None
        if not allow_metal:
            return None, "Ungültiger SMILES"
        mol = Chem.MolFromSmiles(smiles, sanitize=False)
        if mol is None:
            return None, "Ungültiger SMILES (ohne Sanitizing)"
        try:
            Chem.SanitizeMol(
                mol,
                sanitizeOps=(
                    Chem.SanitizeFlags.SANITIZE_ALL
                    ^ Chem.SanitizeFlags.SANITIZE_PROPERTIES
                    ^ Chem.SanitizeFlags.SANITIZE_KEKULIZE
                ),
            )
            return mol, "partielles Sanitizing"
        except Exception:
            return mol, "Sanitizing übersprungen"
    except Exception as e:
        return None, f"RDKit Fehler: {e}"
def smiles_to_xyz(smiles):
    """
    Konvertiere SMILES zu XYZ-Koordinaten (ohne Optimierung).
    Verwendet stk für Metallkomplexe, RDKit für organische Moleküle.
    Returns: (xyz_string, num_atoms, method_used, error_msg)
    """
    has_metal = contains_metal(smiles)
    stk_error = None
    
    # Versuche stk für Metallkomplexe
    if has_metal and HAS_STK:
        try:
            bb = stk.BuildingBlock(smiles)
            mol = bb.to_rdkit_mol()
            if mol.GetNumConformers() == 0:
                AllChem.EmbedMolecule(mol, randomSeed=42, useRandomCoords=True)
            method = "stk"
        except Exception as e:
            # Fallback zu RDKit
            stk_error = str(e)
            mol = None
            method = None
    else:
        mol = None
        method = None
    
    # RDKit als Fallback oder für organische Moleküle
    if mol is None:
        mol, rdkit_note = mol_from_smiles_rdkit(smiles, allow_metal=has_metal)
        if mol is None:
            if stk_error:
                return None, 0, None, f"RDKit: {rdkit_note}; stk: {stk_error}"
            return None, 0, None, rdkit_note
        
        try:
            mol = Chem.AddHs(mol)
        except Exception:
            pass

        try:
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            params.useRandomCoords = True
            try:
                params.maxAttempts = 100
            except Exception:
                pass

            try:
                result = AllChem.EmbedMolecule(mol, params, maxAttempts=50)
            except TypeError:
                result = AllChem.EmbedMolecule(mol, params)
            if result == -1:
                params2 = AllChem.ETKDGv3()
                params2.randomSeed = 42
                params2.useRandomCoords = True
                try:
                    params2.maxAttempts = 200
                except Exception:
                    pass
                try:
                    result = AllChem.EmbedMolecule(mol, params2, maxAttempts=200)
                except TypeError:
                    result = AllChem.EmbedMolecule(mol, params2)
                if result == -1:
                    return None, 0, None, "Konnte keine 3D-Struktur generieren"
            method = "RDKit"
            if rdkit_note:
                method = f"RDKit ({rdkit_note})"
        except Exception as e:
            return None, 0, None, f"RDKit Fehler: {str(e)}"
    
    # XYZ-String erstellen
    try:
        conf = mol.GetConformer()
        num_atoms = mol.GetNumAtoms()
        xyz_lines = []
        for i in range(num_atoms):
            atom = mol.GetAtomWithIdx(i)
            pos = conf.GetAtomPosition(i)
            xyz_lines.append(f"{atom.GetSymbol():2s} {pos.x:12.6f} {pos.y:12.6f} {pos.z:12.6f}")
        
        xyz_string = '\n'.join(xyz_lines)
        return xyz_string, num_atoms, method, None
    except Exception as e:
        return None, 0, None, f"Koordinaten-Fehler: {str(e)}"


def is_smiles(text):
    """Prueft ob der Text ein SMILES-String ist."""
    text = text.strip()
    if '\n' in text:
        return False
    smiles_pattern = r'^[A-Za-z0-9\[\]()=#@+\-\\\/.:,%]+$'
    return bool(re.match(smiles_pattern, text))


def clean_input_data(input_text):
    """
    Verarbeitet Eingabedaten: SMILES oder XYZ-Koordinaten.
    Returns: (cleaned_text, input_type)
    """
    text = input_text.strip()
    
    if not text:
        return '', 'empty'
    
    if is_smiles(text):
        return text, 'smiles'
    
    lines = text.split('\n')
    if len(lines) < 2:
        return text, 'xyz'
    
    first_line = lines[0].strip()
    try:
        atom_count = int(first_line)
        cleaned_lines = lines[2:]
        return '\n'.join(cleaned_lines).strip(), 'xyz'
    except ValueError:
        return text, 'xyz'


def parse_resource_settings(control_text):
    """Parse PAL and maxcore from CONTROL.txt content."""
    pal_match = re.search(r'^\s*PAL\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    maxcore_match = re.search(r'^\s*maxcore\s*=\s*(\d+)', control_text, flags=re.MULTILINE)
    pal = int(pal_match.group(1)) if pal_match else None
    maxcore = int(maxcore_match.group(1)) if maxcore_match else None
    return pal, maxcore


def apply_resources_to_submit_script(script_text, pal, maxcore):
    """Align SBATCH resources with PAL/maxcore if provided."""
    if pal:
        script_text = re.sub(r'(?m)^#SBATCH\s+--ntasks=\d+.*$', f'#SBATCH --ntasks={pal}', script_text)
    if pal and maxcore:
        mem_mb = pal * maxcore
        script_text = re.sub(r'(?m)^#SBATCH\s+--mem=\S+.*$', f'#SBATCH --mem={mem_mb}M', script_text)
    return script_text


DEFAULT_CONTROL = """input_file=input.txt
NAME=
SMILES=
charge=[CHARGE]
------------------------------------
Solvation:
implicit_solvation_model=CPCM
solvent=[SOLVENT]
XTB_SOLVATOR=no
number_explicit_solv_molecules=2
------------------------------------
Global geometry optimisation:
xTB_method=XTB2
XTB_OPT=no
XTB_GOAT=no
CREST=no
multiplicity_global_opt=
------------------------------------
IMAG=yes
IMAG_scope=initial
IMAG_option=2
allow_imaginary_freq=0
IMAG_sp_energy_window=1e-3
IMAG_optimize_candidates=no
------------------------------------
calc_prop_of_interest=no
properties_of_interest=IP,EA
------------------------------------
Redox steps:
calc_initial=yes
oxidation_steps=1,2,3
reduction_steps=1,2,3
method=classic|manually|OCCUPIER
calc_potential_method=2
------------------------------------
ESD module (excited state dynamics):
ESD_modul=no
ESD_modus=TDDFT|deltaSCF|hybrid1
ESD_frequency=yes
states=S1,T1,S2,T2
ISCs=S1>T1,T1>S1
ICs=S2>S1
emission_rates=f,p
phosp_IROOT=1,2,3
phosp_keywords=
fluor_keywords=
TROOTSSL=-1,0,1
addition_S0=
DOHT=TRUE
ESD_LINES=LORENTZ
ESD_LINEW=50
ESD_INLINEW=250
ESD_NPOINTS=131072
ESD_MAXTIME=12000
hybrid1_geom_MaxIter=60
--------------------
Electrical Properties:
elprop_Dipole=no
elprop_Quadrupole=no
elprop_Hyperpol=no
elprop_Polar=no
elprop_PolarVelocity=no
elprop_PolarDipQuad=no
elprop_PolarQuadQuad=no
--------------------
deltaSCF Settings:
deltaSCF_DOMOM=true
deltaSCF_PMOM=true
deltaSCF_keepinitialref=true
deltaSCF_SOSCFHESSUP=LSR1
deltaSCF_keywords=FreezeAndRelease
deltaSCF_maxiter=300
deltaSCF_SOSCFConvFactor=500
deltaSCF_SOSCFMaxStep=0.1
--------------------
TDDFT Settings:
TDDFT_TDDFT_maxiter=500
TDDFT_nroots=15
TDDFT_maxdim=30
TDDFT_TDA=FALSE
TDDFT_followiroot=true
TDDFT_SOC=false
------------------------------------
MANUALLY:
multiplicity_0=
additions_0=
additions_TDDFT=
additions_T1=
additions_S1=
multiplicity_ox1=
additions_ox1=
multiplicity_ox2=
additions_ox2=
multiplicity_ox3=
additions_ox3=
multiplicity_red1=
additions_red1=
multiplicity_red2=
additions_red2=
multiplicity_red3=
additions_red3=
------------------------------------
Level of Theory:
functional=PBE0
disp_corr=D4
ri_jkx=RIJCOSX
relativity=ZORA
aux_jk=def2/J
aux_jk_rel=SARC/J
main_basisset=def2-SVP
main_basisset_rel=ZORA-def2-SVP
metal_basisset=def2-TZVP
metal_basisset_rel=SARC-ZORA-TZVP
first_coordination_sphere_metal_basisset=no
first_coordination_sphere_scale=1.3
geom_opt=OPT
freq_type=FREQ
initial_guess=PModel
temperature=298.15
maxiter=125
qmmm_option=QM/PBEH-3c
------------------------------------
Reference value:
E_ref=
------------------------------------
Literature_reference=
reference_CV=V Vs. Fc+/Fc
E_00_exp=
E_red_exp=
E_red_2_exp=
E_red_3_exp=
E_ox_exp=
E_ox_2_exp=
E_ox_3_exp=
*E_red_exp=
*E_ox_exp=
------------------------------------
Prints:
print_MOs=yes
print_Loewdin_population_analysis=no
------------------------------------
Resource Settings:
PAL=40
maxcore=6000
parallel_workflows=yes
pal_jobs=4
orca_parallel_strategy=auto
enable_job_timeouts=no
job_timeout_hours=36
opt_timeout_hours=14
frequency_timeout_hours=36
sp_timeout_hours=3
------------------------------------
Automatic Error Recovery & Retry:
enable_auto_recovery=yes
max_recovery_attempts=1
enable_adaptive_parallelism=yes
enable_performance_metrics=yes
------------------------------------
OCCUPIER-Settings:
--------------------
OCCUPIER_method=auto
OCCUPIER_tree=own
OWN_TREE_PURE_WINDOW=3
OWN_progressive_from=no
fob_equal_weights=yes
frequency_calculation_OCCUPIER=no
occupier_selection=tolerance
occupier_precision=3
occupier_epsilon=5e-4
clean_override_window_h=0.002
clean_quality_improvement=0.05
clean_quality_good=0.05
maxiter_occupier=125
geom_opt_OCCUPIER=OPT
pass_wavefunction=no
approximate_spin_projection_APMethod=2
--------------------
OCCUPIER_sequence_profiles:
-3,-2,-1,0,+1,+2,+3=[
even electron number:
even_seq = [
  {"index": 1, "m": 1, "BS": "",    "from": 0},
  {"index": 2, "m": 1, "BS": "1,1", "from": 1},
  {"index": 3, "m": 1, "BS": "2,2", "from": 2},
  {"index": 4, "m": 3, "BS": "",    "from": 1},
  {"index": 5, "m": 3, "BS": "3,1", "from": 4},
  {"index": 6, "m": 3, "BS": "4,2", "from": 5},
  {"index": 7, "m": 5, "BS": "",    "from": 4}
]
-------------------
odd electron number:
odd_seq = [
  {"index": 1, "m": 2, "BS": "",    "from": 0},
  {"index": 2, "m": 2, "BS": "2,1", "from": 1},
  {"index": 3, "m": 2, "BS": "3,2", "from": 2},
  {"index": 4, "m": 4, "BS": "",    "from": 1},
  {"index": 5, "m": 4, "BS": "4,1", "from": 4},
  {"index": 6, "m": 4, "BS": "5,2", "from": 5},
  {"index": 7, "m": 6, "BS": "",    "from": 4}
]
]
"""

common_layout = widgets.Layout(width='500px')
common_style = {'description_width': 'initial'}

# === Widgets ===
job_name_widget = widgets.Text(
    value='',
    placeholder='e.g. Fe_Complex_Ox',
    description='Job Name:',
    layout=common_layout,
    style=common_style
)

job_type_widget = widgets.RadioButtons(
    options=['Short (24h)', 'Standard (48h)', 'Long (72h)', 'Extra Long (120h)'],
    value='Standard (48h)',
    description='Job Type:',
    style=common_style,
    layout=widgets.Layout(width='500px')
)

coords_help = widgets.Label("Input: XYZ coordinates (header auto-removed) or SMILES string")
coords_widget = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates or SMILES:\n\nXYZ example:\n42\nComment\nFe  0.0  0.0  0.0\nC   1.5  0.0  0.0\n\nSMILES example:\nCCO or c1ccccc1',
    description='Input:',
    layout=widgets.Layout(width='500px', height='200px'),
    style=common_style
)

# Spacer für konsistenten Abstand vor Buttons (10px)
button_spacer = widgets.Label(value="", layout=widgets.Layout(height="10px"))

convert_smiles_button = widgets.Button(
    description='CONVERT SMILES',
    button_style='info',
    layout=widgets.Layout(width='180px', height='35px')
)

smiles_batch_help = widgets.Label("Batch SMILES list: one per line 'Name;SMILES;key=value;key=value'")
smiles_batch_widget = widgets.Textarea(
    value='',
    placeholder='name;SMILES;key=value;...\nNi_1;[Ni];charge=2;solvent=water\nCo_1;[Co];charge=3',
    description='SMILES List:',
    layout=widgets.Layout(width='500px', height='160px'),
    style=common_style
)

submit_smiles_list_button = widgets.Button(
    description='SUBMIT SMILES LIST',
    button_style='success',
    layout=widgets.Layout(width='220px', height='40px')
)
smiles_batch_output = widgets.Output()

control_help = widgets.Label("CONTROL.txt - edit parameters as needed")
control_widget = widgets.Textarea(
    value=DEFAULT_CONTROL,
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='500px'),
    style=common_style
)

submit_button = widgets.Button(
    description='SUBMIT JOB',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='50px')
)

validate_button = widgets.Button(
    description='VALIDATE CONTROL',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='50px')
)

output_area = widgets.Output()
validate_output = widgets.Output()

mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #1976d2',
    width='600px',
    height='450px',
    overflow='hidden'
))

only_goat_label = widgets.Label("Only GOAT:")
only_goat_charge = widgets.IntText(
    value=0,
    description='Charge:',
    style=common_style,
    layout=widgets.Layout(width='160px')
)
only_goat_solvent = widgets.Text(
    value='',
    placeholder='e.g. water',
    description='Solvent:',
    style=common_style,
    layout=widgets.Layout(width='220px')
)
only_goat_submit_button = widgets.Button(
    description='SUBMIT ONLY GOAT',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)
only_goat_output = widgets.Output()

# Speichere konvertierte XYZ-Koordinaten für Job-Submit
_converted_xyz_cache = {'smiles': None, 'xyz': None}


def update_molecule_view(change=None):
    """Update Molekül-Visualisierung - konvertiert SMILES nicht automatisch."""
    global _converted_xyz_cache

    with mol_output:
        clear_output()
        raw_input = coords_widget.value.strip()

        if not raw_input:
            print("Please enter XYZ coordinates or SMILES.")
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            return

        cleaned_data, input_type = clean_input_data(raw_input)

        if input_type == 'smiles':
            # Avoid heavy conversion on every keystroke; use the convert button instead.
            _converted_xyz_cache = {'smiles': None, 'xyz': None}
            print("SMILES erkannt. Bitte 'CONVERT SMILES' klicken.")
            return

        # XYZ-Koordinaten direkt visualisieren
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        coords = cleaned_data
        lines = [l for l in coords.split('\n') if l.strip()]
        num_atoms = len(lines)
        xyz_data = f"{num_atoms}\nGenerated by widget\n{coords}"
        mol_view = py3Dmol.view(width=590, height=440)
        mol_view.addModel(xyz_data, "xyz")
        mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
        mol_view.zoomTo()
        mol_view.show()


coords_widget.observe(update_molecule_view, names='value')

def handle_convert_smiles(button):
    global _converted_xyz_cache
    raw_input = coords_widget.value.strip()
    if not raw_input:
        with mol_output:
            clear_output()
            print("Please enter SMILES in the input box.")
        return

    with mol_output:
        clear_output()
        print("Converting SMILES...")

    cleaned_data, input_type = clean_input_data(raw_input)
    if input_type != 'smiles':
        with mol_output:
            clear_output()
            print("Input is not a SMILES string.")
        return

    xyz_string, num_atoms, method, error = smiles_to_xyz(cleaned_data)
    if error:
        with mol_output:
            clear_output()
            print(f"SMILES: {cleaned_data}")
            print(f"Fehler: {error}")
        _converted_xyz_cache = {'smiles': None, 'xyz': None}
        return

    # Cache SMILES and XYZ for later use in submit
    _converted_xyz_cache = {'smiles': cleaned_data, 'xyz': xyz_string}
    coords_widget.value = f"{num_atoms}\nConverted from SMILES ({method})\n{xyz_string}"


convert_smiles_button.on_click(handle_convert_smiles)

def handle_submit_smiles_list(button):
    with smiles_batch_output:
        clear_output()

        job_prefix = job_name_widget.value.strip()
        if not job_prefix:
            print("Error: Job name cannot be empty!")
            return

        control_content_base = control_widget.value
        control_errors = validate_control_text(control_content_base)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        job_type = job_type_widget.value
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            return

        entries = [l.strip() for l in smiles_batch_widget.value.splitlines() if l.strip()]
        if not entries:
            print("Error: SMILES list is empty.")
            return

        for idx, entry in enumerate(entries, 1):
            if ';' not in entry:
                print(f"Line {idx}: Missing ';' delimiter -> {entry}")
                continue
            parts = [p.strip() for p in entry.split(';') if p.strip()]
            if len(parts) < 2:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            name_raw = parts[0]
            smiles = parts[1]
            extra_parts = parts[2:]

            if not name_raw or not smiles:
                print(f"Line {idx}: Missing name or SMILES -> {entry}")
                continue

            extras = {}
            for part in extra_parts:
                if '=' not in part:
                    print(f"Line {idx}: Invalid override '{part}' (expected key=value)")
                    continue
                key, value = part.split('=', 1)
                key = key.strip()
                value = value.strip()
                if not key:
                    print(f"Line {idx}: Invalid override '{part}' (empty key)")
                    continue
                extras[key] = value

            safe_name = ''.join(c for c in name_raw if c.isalnum() or c in ('_', '-'))
            if not safe_name:
                print(f"Line {idx}: Invalid name -> {name_raw}")
                continue

            job_name = f"{job_prefix}_{safe_name}"
            safe_job_name = ''.join(c for c in job_name if c.isalnum() or c in ('_', '-'))
            if not safe_job_name:
                print(f"Line {idx}: Invalid job name -> {job_name}")
                continue

            xyz_string, num_atoms, method, error = smiles_to_xyz(smiles)
            if error:
                print(f"Line {idx}: {safe_name} - SMILES error: {error}")
                continue

            job_dir = BASE_DIR / safe_job_name
            job_dir.mkdir(parents=True, exist_ok=True)

            control_content = re.sub(r'(?m)^SMILES=.*$', f"SMILES={smiles}", control_content_base)
            for key, value in extras.items():
                pattern = rf'(?m)^{re.escape(key)}\s*=.*$'
                replacement = f"{key}={value}"
                if re.search(pattern, control_content):
                    control_content = re.sub(pattern, replacement, control_content)
                else:
                    control_content = control_content.rstrip() + f"\n{replacement}\n"

            control_path = job_dir / 'CONTROL.txt'
            control_path.write_text(control_content)

            input_path = job_dir / 'input.txt'
            input_path.write_text(xyz_string)

            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)

            submit_path = job_dir / 'submit_job.sh'
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)

            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else '(unknown)'
                print(f"Submitted {safe_job_name} (ID: {job_id})")
            else:
                print(f"Failed {safe_job_name}: {result.stderr or result.stdout}")


submit_smiles_list_button.on_click(handle_submit_smiles_list)


def reset_form():
    """Setzt das Formular zurueck nach erfolgreichem Submit."""
    global _converted_xyz_cache
    job_name_widget.value = ''
    coords_widget.value = ''
    smiles_batch_widget.value = ''
    control_widget.value = DEFAULT_CONTROL
    job_type_widget.value = 'Standard (48h)'
    only_goat_charge.value = 0
    only_goat_solvent.value = ''
    _converted_xyz_cache = {'smiles': None, 'xyz': None}
    with mol_output:
        clear_output()
        print("Please enter XYZ coordinates or SMILES.")


def handle_submit(button):
    with output_area:
        clear_output()
        
        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        control_content = control_widget.value
        raw_input = coords_widget.value.strip()
        
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return
        
        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return
        
        # Input verarbeiten
        input_content, input_type = clean_input_data(raw_input)
        
        # Bei SMILES: Verwende konvertierte XYZ-Koordinaten
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'
        
        if not input_content:
            print("Error: No valid input found!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename
        
        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            return
        
        try:
            job_dir.mkdir(parents=True, exist_ok=True)
            
            # Write SMILES to CONTROL if available
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )
            
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)
            
            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)
            
            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)
            
            submit_path = job_dir / "submit_job.sh"
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)
            
            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print(f"Job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status: squeue -u $USER")
                
                reset_form()
            else:
                print(f"Error submitting job:")
                print(result.stderr)
        
        except Exception as e:
            print(f"Error creating job: {str(e)}")


def handle_validate_control(button):
    with validate_output:
        clear_output()
        control_content = control_widget.value
        control_errors = validate_control_text(control_content)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
        else:
            print("CONTROL.txt looks valid.")


def handle_only_goat_submit(button):
    with only_goat_output:
        clear_output()

        job_name = job_name_widget.value.strip()
        job_type = job_type_widget.value
        raw_input = coords_widget.value.strip()
        charge_value = only_goat_charge.value
        solvent_value = only_goat_solvent.value.strip()

        if not job_name:
            print("Error: Job name cannot be empty!")
            return

        if not raw_input:
            print("Error: Input (coordinates or SMILES) cannot be empty!")
            return

        if solvent_value == '':
            print("Error: Solvent cannot be empty!")
            return

        input_content, input_type = clean_input_data(raw_input)
        
        # Bei SMILES: Verwende konvertierte XYZ-Koordinaten
        if input_type == 'smiles' and _converted_xyz_cache['xyz']:
            input_content = _converted_xyz_cache['xyz']
            input_type = 'xyz (from SMILES)'

        if not input_content:
            print("Error: No valid input found!")
            return

        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return

        if not ONLY_GOAT_TEMPLATE_PATH.exists():
            print(f"Error: Only GOAT template not found: {ONLY_GOAT_TEMPLATE_PATH}")
            return

        job_dir = BASE_DIR / safe_job_name
        template_filename = JOB_TYPE_TEMPLATES.get(job_type, 'submit_standard.sh')
        template_path = SUBMIT_TEMPLATES_DIR / template_filename

        if not template_path.exists():
            print(f"Error: Template not found: {template_path}")
            return

        try:
            job_dir.mkdir(parents=True, exist_ok=True)

            control_template = ONLY_GOAT_TEMPLATE_PATH.read_text()
            control_content = (control_template
                .replace('[CHARGE]', str(charge_value))
                .replace('[SOLVENT]', solvent_value)
            )
            
            # Write SMILES to CONTROL if available
            if _converted_xyz_cache.get('smiles'):
                control_content = control_content.replace(
                    'SMILES=',
                    f"SMILES={_converted_xyz_cache['smiles']}"
                )
            
            control_errors = validate_control_text(control_content)
            if control_errors:
                print("CONTROL.txt validation failed:")
                for err in control_errors:
                    print(f"- {err}")
                return
            control_path = job_dir / "CONTROL.txt"
            control_path.write_text(control_content)

            input_path = job_dir / "input.txt"
            input_path.write_text(input_content)

            template_content = template_path.read_text()
            submit_script = template_content.replace('{job_name}', safe_job_name)
            pal, maxcore = parse_resource_settings(control_content)
            submit_script = apply_resources_to_submit_script(submit_script, pal, maxcore)

            submit_path = job_dir / "submit_job.sh"
            submit_path.write_text(submit_script)
            submit_path.chmod(0o755)

            result = subprocess.run(
                ['sbatch', 'submit_job.sh'],
                cwd=job_dir,
                capture_output=True,
                text=True
            )

            if result.returncode == 0:
                job_id = result.stdout.strip().split()[-1]
                print("Only GOAT job successfully submitted!")
                print(f"Job ID: {job_id}")
                print(f"Job Type: {job_type}")
                print(f"Input Type: {input_type.upper()}")
                print(f"Charge: {charge_value}")
                print(f"Solvent: {solvent_value}")
                print(f"Directory: {job_dir}")
                print(f"")
                print(f"Check status: squeue -u $USER")

                reset_form()
            else:
                print("Error submitting job:")
                print(result.stderr)

        except Exception as e:
            print(f"Error creating job: {str(e)}")


only_goat_submit_button.on_click(handle_only_goat_submit)
validate_button.on_click(handle_validate_control)
submit_button.on_click(handle_submit)

spacer = widgets.Label(value="", layout=widgets.Layout(height='20px'))

ui_left = widgets.VBox([
    job_name_widget,
    spacer,
    job_type_widget,
    spacer,
    coords_help,
    coords_widget,
    button_spacer,
    convert_smiles_button,
    spacer,
    smiles_batch_help,
    smiles_batch_widget,
    button_spacer,
    submit_smiles_list_button,
    smiles_batch_output,
    spacer,
    control_help,
    control_widget,
    button_spacer,
    validate_button,
    submit_button,
    output_area,
    validate_output
], layout=widgets.Layout(width='55%', padding='10px'))

ui_right = widgets.VBox([
    widgets.Label("Molecule visualization:"),
    mol_output,
    spacer,
    only_goat_label,
    widgets.HBox([
        only_goat_charge,
        only_goat_solvent,
        only_goat_submit_button
    ], layout=widgets.Layout(width='600px')),
    only_goat_output
], layout=widgets.Layout(width='45%', padding='10px'))

full_ui = widgets.HBox([ui_left, ui_right])
tab1_content = full_ui

# === DELFIN RECALC MODULES ===
import re
import subprocess
from IPython.display import display, clear_output
import ipywidgets as widgets

recalc_title = widgets.HTML("<h3>DELFIN RECALC MODULES</h3>")
recalc_help = widgets.Label("Select a job folder from calc, edit CONTROL.txt, then submit a recalc job.")

recalc_folder_dropdown = widgets.Dropdown(
    options=[],
    description='Job Folder:',
    layout=widgets.Layout(width='500px'),
    style={'description_width': 'initial'}
)

recalc_control_widget = widgets.Textarea(
    value='',
    description='CONTROL.txt:',
    layout=widgets.Layout(width='500px', height='400px'),
    style={'description_width': 'initial'}
)

recalc_spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))

recalc_button = widgets.Button(
    description='SUBMIT RECALC',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)
recalc_output = widgets.Output()


def refresh_recalc_folders():
    if not BASE_DIR.exists():
        recalc_folder_dropdown.options = []
        recalc_folder_dropdown.value = None
        return

    folders = sorted([p.name for p in BASE_DIR.iterdir() if p.is_dir()])
    recalc_folder_dropdown.options = folders
    recalc_folder_dropdown.value = None


def load_recalc_control(change=None):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            recalc_control_widget.value = ''
            return

        control_path = BASE_DIR / folder / 'CONTROL.txt'
        if control_path.exists():
            recalc_control_widget.value = control_path.read_text()
        else:
            recalc_control_widget.value = ''
            print(f"CONTROL.txt not found in {control_path.parent}")


def _patch_submit_for_recalc(script_text: str) -> tuple[str, bool]:
    """
    Replace the first occurrence of a delfin invocation with 'delfin --recalc'.
    Handles:
      - a standalone line: 'delfin'
      - a line starting with: 'delfin ...'
    Keeps the rest of the line (args) if present, but ensures --recalc is present.
    """
    lines = script_text.splitlines(True)  # keep line endings

    for i, line in enumerate(lines):
        # Match lines where the command starts with "delfin" (optionally with args)
        m = re.match(r'^(\s*)delfin(\s+.*)?(\s*)$', line)
        if not m:
            continue

        indent, args, trail = m.group(1), (m.group(2) or ''), m.group(3)

        # If --recalc already present, we consider it patched
        if '--recalc' in args:
            return script_text, True

        # Insert --recalc right after delfin, keep any existing args
        new_line = f"{indent}delfin --recalc{args}{trail}"
        lines[i] = new_line
        return ''.join(lines), True

    return script_text, False


def handle_recalc(button):
    with recalc_output:
        clear_output()

        folder = recalc_folder_dropdown.value
        if not folder:
            print("Error: Please select a job folder.")
            return

        job_dir = BASE_DIR / folder
        if not job_dir.exists():
            print(f"Error: Job folder does not exist: {job_dir}")
            return

        # Write updated CONTROL.txt
        control_errors = validate_control_text(recalc_control_widget.value)
        if control_errors:
            print("CONTROL.txt validation failed:")
            for err in control_errors:
                print(f"- {err}")
            return

        control_path = job_dir / 'CONTROL.txt'
        control_path.write_text(recalc_control_widget.value)

        # Use existing submit_job.sh and switch to recalc
        submit_path = job_dir / 'submit_job.sh'
        if not submit_path.exists():
            print(f"Error: submit_job.sh not found in {job_dir}")
            return

        submit_content = submit_path.read_text()
        pal, maxcore = parse_resource_settings(recalc_control_widget.value)
        submit_content = apply_resources_to_submit_script(submit_content, pal, maxcore)
        new_content, replaced = _patch_submit_for_recalc(submit_content)

        if not replaced:
            print("Error: Could not find a 'delfin' command in submit_job.sh")
            return

        submit_path.write_text(new_content)

        result = subprocess.run(
            ['sbatch', 'submit_job.sh'],
            cwd=job_dir,
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            # sbatch output typically: "Submitted batch job <id>"
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("Recalc job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Directory: {job_dir}")
        else:
            print("Error submitting recalc job:")
            print(result.stderr or result.stdout)


recalc_folder_dropdown.observe(load_recalc_control, names='value')
refresh_recalc_folders()

recalc_section = widgets.VBox([
    recalc_title,
    recalc_help,
    recalc_folder_dropdown,
    recalc_control_widget,
    recalc_spacer,
    recalc_button,
    recalc_output
], layout=widgets.Layout(width='600px', padding='10px'))

recalc_button.on_click(handle_recalc)

tab2_content = recalc_section

# === Interactive Job Status ===
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import subprocess
import os

job_status_title = widgets.HTML("<h3>Job Status</h3>")
job_status_help = widgets.HTML("<p>Select a job from the dropdown and click CANCEL JOB to cancel it. Click REFRESH to update the list.</p>")

# Speichere Job-Daten
_job_data = []

# HTML-Tabelle für Job-Anzeige
job_table_html = widgets.HTML(value="<i>Loading...</i>")

# Startzeit-Anzeige für wartende Jobs
job_start_html = widgets.HTML(value="")

# Dropdown für Job-Auswahl
job_dropdown = widgets.Dropdown(
    options=[],
    description='Select Job:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '80px'}
)

job_status_output = widgets.Output()

def refresh_job_list(button=None):
    """Refresh the job list from squeue."""
    global _job_data
    
    with job_status_output:
        clear_output()
    
    try:
        # Formatierte squeue Ausgabe für bessere Spalten
        result = subprocess.run(
            ['squeue', '-u', os.environ.get('USER', ''), 
             '-o', '%.12i %.12P %.35j %.10u %.3t %.12M %.6D %R'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        # Startzeit für wartende Jobs abrufen
        start_result = subprocess.run(
            ['squeue', '-u', os.environ.get('USER', ''), '--start',
             '-o', '%.12i %.35j %.20S'],
            capture_output=True,
            text=True,
            timeout=10
        )
        
        if result.returncode == 0:
            lines = result.stdout.strip().split('\n')
            
            if len(lines) < 2:
                job_table_html.value = "<p><i>No jobs in queue.</i></p>"
                job_start_html.value = ""
                job_dropdown.options = []
                _job_data = []
                return
            
            header = lines[0]
            job_lines = lines[1:]
            
            # Parse jobs
            _job_data = []
            dropdown_options = []
            
            for line in job_lines:
                if not line.strip():
                    continue
                parts = line.split()
                if len(parts) >= 1:
                    job_id = parts[0].strip()
                    job_name = parts[2].strip() if len(parts) > 2 else "unknown"
                    _job_data.append({'id': job_id, 'line': line, 'name': job_name})
                    dropdown_options.append((f"{job_id} - {job_name}", job_id))
            
            # Build HTML table
            table_html = """
            <style>
                .job-table { font-family: monospace; font-size: 12px; border-collapse: collapse; width: 100%; }
                .job-table th, .job-table td { padding: 6px 10px; text-align: left; border-bottom: 1px solid #ddd; }
                .job-table th { background-color: #2196F3; color: white; }
                .job-table tr:hover { background-color: #f5f5f5; }
            </style>
            <table class="job-table">
                <tr>
                    <th>JOBID</th>
                    <th>PARTITION</th>
                    <th>NAME</th>
                    <th>USER</th>
                    <th>ST</th>
                    <th>TIME</th>
                    <th>NODES</th>
                    <th>NODELIST/REASON</th>
                </tr>
            """
            
            for job in _job_data:
                parts = job['line'].split(None, 7)  # Max 8 parts
                while len(parts) < 8:
                    parts.append('')
                table_html += f"""
                <tr>
                    <td><b>{parts[0]}</b></td>
                    <td>{parts[1]}</td>
                    <td>{parts[2]}</td>
                    <td>{parts[3]}</td>
                    <td>{parts[4]}</td>
                    <td>{parts[5]}</td>
                    <td>{parts[6]}</td>
                    <td>{parts[7]}</td>
                </tr>
                """
            
            table_html += "</table>"
            job_table_html.value = table_html
            
            # Parse and display start times for pending jobs
            if start_result.returncode == 0:
                start_lines = start_result.stdout.strip().split('\n')
                if len(start_lines) > 1:
                    pending_jobs = []
                    for line in start_lines[1:]:
                        if line.strip() and 'N/A' not in line:
                            parts = line.split(None, 2)
                            if len(parts) >= 3:
                                pending_jobs.append({
                                    'id': parts[0].strip(),
                                    'name': parts[1].strip(),
                                    'start': parts[2].strip()
                                })
                    
                    if pending_jobs:
                        start_html = """
                        <div style="margin-top: 10px; padding: 10px; background-color: #fff3cd; border: 1px solid #ffc107; border-radius: 4px;">
                            <b>Estimated Start Times (pending jobs):</b><br>
                            <table style="font-family: monospace; font-size: 12px; margin-top: 5px;">
                        """
                        for pj in pending_jobs:
                            start_html += f"<tr><td><b>{pj['id']}</b></td><td style='padding-left:15px;'>{pj['name']}</td><td style='padding-left:15px;'>{pj['start']}</td></tr>"
                        start_html += "</table></div>"
                        job_start_html.value = start_html
                    else:
                        job_start_html.value = ""
                else:
                    job_start_html.value = ""
            else:
                job_start_html.value = ""
            
            # Update dropdown
            if dropdown_options:
                job_dropdown.options = dropdown_options
                job_dropdown.value = None
            else:
                job_dropdown.options = []
            
            with job_status_output:
                clear_output()
                print(f"{len(_job_data)} job(s) found")
        else:
            with job_status_output:
                clear_output()
                print(f"Error: {result.stderr}")
    except Exception as e:
        with job_status_output:
            clear_output()
            print(f"Error fetching jobs: {e}")

def cancel_selected_job(button):
    """Cancel the currently selected job."""
    with job_status_output:
        clear_output()
        
        if not job_dropdown.value:
            print("No job selected.")
            return
        
        job_id = job_dropdown.value
        
        result = subprocess.run(
            ['scancel', str(job_id)],
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print(f"Job {job_id} cancelled successfully.")
            # Refresh the list
            refresh_job_list()
        else:
            print(f"Error cancelling job {job_id}:")
            print(result.stderr or result.stdout)

# Buttons
refresh_button = widgets.Button(
    description='REFRESH',
    button_style='info',
    layout=widgets.Layout(width='120px', height='40px')
)

cancel_button = widgets.Button(
    description='CANCEL JOB',
    button_style='danger',
    layout=widgets.Layout(width='150px', height='40px')
)

refresh_button.on_click(refresh_job_list)
cancel_button.on_click(cancel_selected_job)

# Initial load
refresh_job_list()

# Layout
button_row = widgets.HBox([
    job_dropdown,
    cancel_button,
    refresh_button
], layout=widgets.Layout(margin='10px 0', align_items='center'))

job_status_section = widgets.VBox([
    job_status_title,
    job_status_help,
    job_table_html,
    job_start_html,
    button_row,
    job_status_output
], layout=widgets.Layout(width='600px', padding='10px', border='1px solid #2196F3'))

tab3_content = job_status_section

# === ORCA Input Builder ===
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess
import re
from pathlib import Path
from delfin.common.control_validator import ORCA_FUNCTIONALS, ORCA_BASIS_SETS, DISP_CORR_VALUES, _RI_JKX_KEYWORDS

# Ensure BASE_DIR is available even if the main dashboard cell wasn't run
try:
    BASE_DIR
except NameError:
    if 'ROOT_DIR' in globals():
        BASE_DIR = ROOT_DIR / 'calc_orca'
    else:
        BASE_DIR = Path.cwd() / 'calc_orca'



orca_method_options = sorted(ORCA_FUNCTIONALS)
orca_basis_options = sorted(ORCA_BASIS_SETS)
orca_dispersion_options = ['None'] + sorted(v for v in DISP_CORR_VALUES if v)
orca_ri_options = ['None'] + sorted(_RI_JKX_KEYWORDS)
orca_aux_basis_options = ['None', 'def2/J', 'def2/JK']

orca_builder_title = widgets.HTML("<h3>ORCA Input Builder</h3>")
orca_builder_help = widgets.Label("Build ORCA input files and submit standalone ORCA calculations.")

# === ORCA Builder Widgets ===
orca_job_name = widgets.Text(
    value='',
    placeholder='e.g. water_opt',
    description='Job Name:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '120px'}
)

orca_coords = widgets.Textarea(
    value='',
    placeholder='Paste XYZ coordinates (with or without header):\n\n6\nComment line\nC  0.0  0.0  0.0\n...\n\nor just:\nC  0.0  0.0  0.0\n...:\nO   0.000000   0.000000   0.117300\nH   0.000000   0.756950  -0.469200\nH   0.000000  -0.756950  -0.469200',
    description='Coordinates:',
    layout=widgets.Layout(width='650px', height='400px'),
    style={'description_width': '120px'}
)

orca_charge = widgets.IntText(
    value=0,
    description='Charge:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_multiplicity = widgets.IntText(
    value=1,
    description='Multiplicity:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_method = widgets.Dropdown(
    options=orca_method_options,
    value='PBE0',
    description='Method:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_job_type = widgets.Dropdown(
    options=['SP', 'OPT', 'FREQ', 'OPT FREQ'],
    value='OPT',
    description='Job Type:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_basis = widgets.Dropdown(
    options=orca_basis_options,
    value='def2-SVP',
    description='Basis Set:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_dispersion = widgets.Dropdown(
    options=orca_dispersion_options,
    value='D4',
    description='Dispersion:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_ri = widgets.Dropdown(
    options=orca_ri_options,
    value='RIJCOSX',
    description='RI Approx:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

orca_aux_basis = widgets.Dropdown(
    options=orca_aux_basis_options,
    value='def2/J',
    description='Aux Basis:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)

# === Solvation (CPCM) ===
orca_cpcm_enabled = widgets.Checkbox(
    value=False,
    description='CPCM',
    layout=widgets.Layout(width='80px'),
    style={'description_width': 'initial'}
)

orca_solvent = widgets.Dropdown(
    options=['water', 'acetonitrile', 'dmso', 'dmf', 'methanol', 'ethanol', 'thf', 'dichloromethane', 'chloroform', 'toluene', 'hexane'],
    value='water',
    description='Solvent:',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '70px'}
)

# === Print options ===
orca_print_mos = widgets.Checkbox(
    value=False,
    description='Print MOs',
    layout=widgets.Layout(width='120px'),
    style={'description_width': 'initial'}
)

orca_print_basis = widgets.Checkbox(
    value=False,
    description='Print Basis',
    layout=widgets.Layout(width='120px'),
    style={'description_width': 'initial'}
)



orca_additional = widgets.Text(
    value='',
    placeholder='e.g. FinalGrid6 NormalPrint',
    description='Additional:',
    layout=widgets.Layout(width='400px'),
    style={'description_width': '120px'}
)

orca_pal = widgets.IntText(
    value=40,
    description='PAL (cores):',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_maxcore = widgets.IntText(
    value=6000,
    description='MaxCore (MB):',
    layout=widgets.Layout(width='200px'),
    style={'description_width': '120px'}
)

orca_slurm_time = widgets.Text(
    value='12:00:00',
    placeholder='e.g. 02:00:00',
    description='Time Limit:',
    layout=widgets.Layout(width='250px'),
    style={'description_width': '120px'}
)



# === File Upload (Drag & Drop) ===
orca_file_upload = widgets.FileUpload(
    accept='',
    multiple=True,
    description='Extra Files:',
    layout=widgets.Layout(width='500px', height='80px'),
    style={'description_width': '120px'}
)

orca_uploaded_files_label = widgets.HTML(
    value='<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>',
    layout=widgets.Layout(width='500px', height='80px')
)

# Preview area
orca_preview = widgets.Textarea(
    value='',
    description='INP Preview:',
    layout=widgets.Layout(width='750px', height='550px'),
    style={'description_width': '120px'},
    disabled=False
)

# Buttons
orca_generate_btn = widgets.Button(
    description='GENERATE INP',
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px')
)

orca_save_btn = widgets.Button(
    description='SAVE INP + SH',
    button_style='warning',
    layout=widgets.Layout(width='150px', height='40px')
)

orca_submit_btn = widgets.Button(
    description='SUBMIT ORCA JOB',
    button_style='success',
    layout=widgets.Layout(width='180px', height='40px')
)

orca_output = widgets.Output()

orca_mol_output = widgets.Output(layout=widgets.Layout(
    border='2px solid #4CAF50',
    width='600px',
    height='450px',
    overflow='hidden'
))


def reset_orca_builder():
    """Reset all ORCA Builder fields to their default values."""
    orca_job_name.value = ''
    orca_coords.value = ''
    orca_charge.value = 0
    orca_multiplicity.value = 1
    orca_method.value = 'PBE0'
    orca_job_type.value = 'OPT'
    orca_basis.value = 'def2-SVP'
    orca_dispersion.value = 'D4'
    orca_ri.value = 'RIJCOSX'
    orca_aux_basis.value = 'def2/J'
    orca_cpcm_enabled.value = False
    orca_solvent.value = 'water'
    orca_print_mos.value = False
    orca_print_basis.value = False
    orca_additional.value = ''
    orca_pal.value = 40
    orca_maxcore.value = 6000
    orca_slurm_time.value = '12:00:00'
    orca_file_upload.value = ()  # Clear uploaded files
    orca_uploaded_files_label.value = ''
    update_orca_preview()



def strip_xyz_header(text):
    """Remove XYZ header (atom count + comment line) if present."""
    text = text.strip()
    if not text:
        return text
    
    lines = text.split('\n')
    if len(lines) < 2:
        return text
    
    first_line = lines[0].strip()
    try:
        int(first_line)  # If first line is a number, it's an atom count
        return '\n'.join(lines[2:]).strip()  # Skip first 2 lines
    except ValueError:
        return text  # No header, return as-is


def update_orca_molecule_view(change=None):
    """Update molecule visualization for ORCA Builder."""
    with orca_mol_output:
        clear_output()
        raw_input = orca_coords.value.strip()
        
        if not raw_input:
            print("Paste XYZ coordinates to see 3D preview.")
            return
        
        # Strip header if present
        coords = strip_xyz_header(raw_input)
        
        if not coords:
            print("No valid coordinates.")
            return
        
        try:
            lines = [l for l in coords.split('\n') if l.strip()]
            num_atoms = len(lines)
            xyz_data = f"{num_atoms}\nORCA Builder Preview\n{coords}"
            mol_view = py3Dmol.view(width=590, height=440)
            mol_view.addModel(xyz_data, "xyz")
            mol_view.setStyle({}, {"stick": {"radius": 0.15}, "sphere": {"scale": 0.22}})
            mol_view.zoomTo()
            mol_view.show()
        except Exception as e:
            print(f"Could not visualize: {e}")


def generate_orca_input():
    """Generate ORCA input file content from widget values."""
    keywords = []
    
    # Method
    keywords.append(orca_method.value)
    
    # Job type
    keywords.append(orca_job_type.value)
    
    # Basis set
    keywords.append(orca_basis.value)
    
    # Dispersion
    if orca_dispersion.value != 'None':
        keywords.append(orca_dispersion.value)
    
    # RI approximation
    if orca_ri.value != 'None':
        keywords.append(orca_ri.value)
    
    # Auxiliary basis
    if orca_ri.value != 'None':
        keywords.append(orca_aux_basis.value)
    
    # CPCM Solvation - simple syntax: CPCM(solvent)
    if orca_cpcm_enabled.value:
        keywords.append(f'CPCM({orca_solvent.value})')
    
    # Output block
    output_lines = []
    if orca_print_mos.value:
        output_lines.append("  print[p_mos] 1")
    if orca_print_basis.value:
        output_lines.append("  print[p_basis] 2")
    output_block = ""
    if output_lines:
        output_block = "%output\n" + "\n".join(output_lines) + "\nend"
    
    # Additional keywords
    if orca_additional.value.strip():
        keywords.append(orca_additional.value.strip())
    
    keyword_line = '! ' + ' '.join(keywords)
    
    # PAL block
    pal_block = f"""%pal
  nprocs {orca_pal.value}
end"""
    
    # MaxCore
    maxcore_line = f"%maxcore {orca_maxcore.value}"
    
    # Coordinates block
    coords = strip_xyz_header(orca_coords.value)
    coord_block = f"""* xyz {orca_charge.value} {orca_multiplicity.value}
{coords}
*"""
    
    # Assemble full input
    inp_content = f"""{keyword_line}

{pal_block}

{maxcore_line}
"""
    if output_block:
        inp_content += f"\n{output_block}\n"
    inp_content += f"\n{coord_block}\n"
    return inp_content

def parse_inp_resources(inp_text):
    """Parse PAL (nprocs) and maxcore from ORCA input text."""
    pal = None
    maxcore = None
    if not inp_text:
        return pal, maxcore
    pal_match = re.search(r'(?im)^\s*nprocs\s+(\d+)', inp_text)
    if pal_match:
        pal = int(pal_match.group(1))
    maxcore_match = re.search(r'(?im)^\s*%maxcore\s+(\d+)', inp_text)
    if maxcore_match:
        maxcore = int(maxcore_match.group(1))
    return pal, maxcore


def _patch_submit_for_orca(script_text: str, job_name: str) -> tuple[str, bool]:
    """Replace a delfin invocation with an ORCA run command."""
    lines = script_text.splitlines(True)
    for i, line in enumerate(lines):
        m = re.match(r'^(\s*)delfin(\s+.*)?(\s*)$', line)
        if not m:
            continue
        indent, trail = m.group(1), m.group(3)
        lines[i] = f"{indent}delfin run_orca --input {job_name}.inp --output {job_name}.out{trail}"
        return ''.join(lines), True
    return script_text, False


def generate_submit_script(job_name, inp_text):
    """Generate submit_orca.sh script content from cluster templates."""
    pal = orca_pal.value
    maxcore = orca_maxcore.value
    inp_pal, inp_maxcore = parse_inp_resources(inp_text)
    if inp_pal is not None:
        pal = inp_pal
    if inp_maxcore is not None:
        maxcore = inp_maxcore
    total_mem_mb = pal * maxcore if pal and maxcore else orca_pal.value * orca_maxcore.value
    template_filename = 'submit_standard.sh'

    template_path = None
    if 'SUBMIT_TEMPLATES_DIR' in globals():
        template_path = Path(SUBMIT_TEMPLATES_DIR) / template_filename
    else:
        submit_dir = None
        for base in [Path.cwd(), *Path.cwd().parents]:
            cand = base / 'software' / 'delfin' / 'examples' / 'example_Job_Submission_Scripts' / 'BwUniCluster' / 'submit_sh'
            if cand.exists():
                submit_dir = cand
                break
        if submit_dir is not None:
            template_path = submit_dir / template_filename
    if template_path is None or not template_path.exists():
        raise FileNotFoundError(f"{template_filename} not found in submit_sh")

    template_content = template_path.read_text()
    submit_script = template_content.replace('{job_name}', job_name)
    submit_script = re.sub(r'(?m)^#SBATCH\s+--ntasks=\d+.*$', f'#SBATCH --ntasks={pal}', submit_script)
    submit_script = re.sub(r'(?m)^#SBATCH\s+--mem=\S+.*$', f'#SBATCH --mem={total_mem_mb}M', submit_script)
    submit_script = re.sub(r'(?m)^#SBATCH\s+--time=\S+.*$', f'#SBATCH --time={orca_slurm_time.value}', submit_script)
    submit_script, replaced = _patch_submit_for_orca(submit_script, job_name)
    if not replaced:
        raise ValueError("Could not find a 'delfin' command in submit template")
    return submit_script


def save_uploaded_files(job_dir):
    """Save uploaded files to job directory."""
    saved_files = []
    if orca_file_upload.value:
        for uploaded_file in orca_file_upload.value:
            filename = uploaded_file['name'] if isinstance(uploaded_file, dict) else uploaded_file.name
            content = uploaded_file['content'] if isinstance(uploaded_file, dict) else uploaded_file.content
            file_path = job_dir / filename
            file_path.write_bytes(content)
            saved_files.append(filename)
    return saved_files


def update_uploaded_files_label(change=None):
    """Update the label showing uploaded files."""
    if orca_file_upload.value:
        filenames = []
        for f in orca_file_upload.value:
            name = f['name'] if isinstance(f, dict) else f.name
            filenames.append(name)
        orca_uploaded_files_label.value = f"<b>Files to upload:</b> {', '.join(filenames)}"
    else:
        orca_uploaded_files_label.value = '<i>Drag & drop files here (e.g. .gbw, .xyz, .hess)</i>'


orca_file_upload.observe(update_uploaded_files_label, names='value')


def update_orca_preview(change=None):
    """Update the ORCA input preview."""
    orca_preview.value = generate_orca_input()


def handle_orca_generate(button):
    """Handle GENERATE INP button click."""
    update_orca_preview()
    with orca_output:
        clear_output()
        print("ORCA input generated. You can edit the preview if needed.")


def handle_orca_save(button):
    """Handle SAVE INP + SH button click."""
    with orca_output:
        clear_output()
        
        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)
        
        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()
        
        # Save .inp file
        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)
        
        # Save submit_orca.sh
        submit_script = generate_submit_script(safe_job_name, inp_content)
        submit_path = job_dir / "submit_orca.sh"
        submit_path.write_text(submit_script)
        submit_path.chmod(0o755)
        
        # Save uploaded files
        saved_files = save_uploaded_files(job_dir)
        
        # Submit job
        result = subprocess.run(
            ['sbatch', 'submit_orca.sh'],
            cwd=job_dir,
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_cpcm_enabled.value:
                print(f"Solvation: CPCM({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status: squeue -u $USER")
            # Reset form for next job
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


def handle_orca_submit(button):
    """Handle SUBMIT ORCA JOB button click."""
    with orca_output:
        clear_output()
        
        job_name = orca_job_name.value.strip()
        if not job_name:
            print("Error: Job name cannot be empty!")
            return
        
        preview_content = orca_preview.value.strip()
        if preview_content:
            inp_content = preview_content
        else:
            coords = strip_xyz_header(orca_coords.value)
            if not coords:
                print("Error: Coordinates or INP preview cannot be empty!")
                return
            inp_content = generate_orca_input()
        
        safe_job_name = "".join(c for c in job_name if c.isalnum() or c in ('_', '-'))
        if not safe_job_name:
            print("Error: Job name contains only invalid characters!")
            return
        
        job_dir = BASE_DIR / safe_job_name
        job_dir.mkdir(parents=True, exist_ok=True)
        
        # Save .inp file
        inp_path = job_dir / f"{safe_job_name}.inp"
        inp_path.write_text(inp_content)
        
        # Save submit_orca.sh
        submit_script = generate_submit_script(safe_job_name, inp_content)
        submit_path = job_dir / "submit_orca.sh"
        submit_path.write_text(submit_script)
        submit_path.chmod(0o755)
        
        # Save uploaded files
        saved_files = save_uploaded_files(job_dir)
        
        # Submit job
        result = subprocess.run(
            ['sbatch', 'submit_orca.sh'],
            cwd=job_dir,
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            job_id = result.stdout.strip().split()[-1] if result.stdout.strip() else "(unknown)"
            print("ORCA job successfully submitted!")
            print(f"Job ID: {job_id}")
            print(f"Job Name: {safe_job_name}")
            print(f"Directory: {job_dir}")
            pal_used, maxcore_used = parse_inp_resources(inp_content)
            if pal_used is None:
                pal_used = orca_pal.value
            if maxcore_used is None:
                maxcore_used = orca_maxcore.value
            print(f"PAL: {pal_used}, MaxCore: {maxcore_used} MB")
            print(f"Time Limit: {orca_slurm_time.value}")
            if orca_cpcm_enabled.value:
                print(f"Solvation: CPCM({orca_solvent.value})")
            if saved_files:
                print(f"Extra files: {', '.join(saved_files)}")
            print()
            print("Check status: squeue -u $USER")
            # Reset form for next job
            reset_orca_builder()
        else:
            print("Error submitting job:")
            print(result.stderr or result.stdout)


# Connect buttons
orca_generate_btn.on_click(handle_orca_generate)
orca_save_btn.on_click(handle_orca_save)
orca_submit_btn.on_click(handle_orca_submit)

# Connect widget changes to preview update
for w in [orca_method, orca_job_type, orca_basis, orca_dispersion, orca_ri, 
          orca_aux_basis, orca_charge, orca_multiplicity, orca_pal, orca_maxcore,
          orca_coords, orca_additional, orca_cpcm_enabled, orca_solvent,
          orca_print_mos, orca_print_basis, orca_slurm_time]:
    w.observe(update_orca_preview, names='value')

# Initial preview

# Connect coords to molecule viewer
orca_coords.observe(update_orca_molecule_view, names='value')

# Initial molecule view
update_orca_molecule_view()

update_orca_preview()

# Layout
orca_row1 = widgets.HBox([orca_job_name])
orca_row2 = widgets.HBox([orca_coords])
orca_row3 = widgets.HBox([orca_charge, orca_multiplicity])
orca_row4 = widgets.HBox([orca_method, orca_job_type])
orca_row5 = widgets.HBox([orca_basis, orca_dispersion])
orca_row6 = widgets.HBox([orca_ri, orca_aux_basis])
orca_row_solv = widgets.HBox([orca_cpcm_enabled, orca_solvent])
orca_row_opts = widgets.HBox([orca_print_mos, orca_print_basis])
orca_row7 = widgets.HBox([orca_additional])
orca_row8 = widgets.HBox([orca_pal, orca_maxcore])
orca_row9 = widgets.HBox([orca_slurm_time])
orca_row_upload = widgets.VBox([orca_file_upload, orca_uploaded_files_label])
orca_row_buttons = widgets.HBox([orca_generate_btn, orca_save_btn, orca_submit_btn])

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_right = widgets.VBox([
    widgets.Label("Live ORCA Input Preview (editable):"),
    orca_preview,
    widgets.HTML('<br><b>Molecule Preview:</b>'),
    orca_mol_output
], layout=widgets.Layout(width='50%', padding='10px'))

orca_builder_ui = widgets.HBox([orca_left, orca_right])

orca_builder_section = widgets.VBox([
    orca_builder_title,
    orca_builder_help,
    orca_builder_ui
], layout=widgets.Layout(width='600px', padding='10px', border='1px solid #4CAF50'))

tab4_content = orca_builder_section


# === Layout (Dashboard6 style) ===
spacer = widgets.Label(value='', layout=widgets.Layout(height='10px'))
spacer_large = widgets.Label(value='', layout=widgets.Layout(height='20px'))

submit_left = widgets.VBox([
    job_name_widget, spacer, job_type_widget, spacer_large,
    widgets.HTML('<b>Input (XYZ or SMILES):</b>'), coords_widget, spacer, convert_smiles_button,
    spacer_large, widgets.HTML('<b>Batch SMILES:</b>'), smiles_batch_widget, spacer, submit_smiles_list_button, smiles_batch_output,
    spacer_large, widgets.HTML('<b>CONTROL.txt:</b>'), control_widget, spacer,
    widgets.HBox([validate_button, submit_button]), output_area, validate_output
], layout=widgets.Layout(width='50%', padding='10px'))

submit_right = widgets.VBox([
    widgets.HTML('<b>Molecule Preview:</b>'), mol_output,
    spacer_large, widgets.HTML('<b>Only GOAT:</b>'),
    widgets.HBox([only_goat_charge, only_goat_solvent, only_goat_submit_button]),
    only_goat_output
], layout=widgets.Layout(width='50%', padding='10px'))

tab1_content = widgets.HBox([submit_left, submit_right])

recalc_refresh_btn = widgets.Button(description='REFRESH FOLDERS', button_style='info', layout=widgets.Layout(width='150px', height='35px'))
recalc_refresh_btn.on_click(lambda b: refresh_recalc_folders())

tab2_content = widgets.VBox([
    widgets.HTML('<h3>Recalc DELFIN Job</h3>'),
    widgets.HTML('<p>Select a job folder, edit CONTROL.txt, and resubmit.</p>'),
    widgets.HBox([recalc_folder_dropdown, recalc_refresh_btn]),
    recalc_control_widget, spacer,
    recalc_button, recalc_output
], layout=widgets.Layout(padding='10px'))

tab3_content = widgets.VBox([
    widgets.HTML('<h3>Job Status</h3>'),
    widgets.HTML('<p>Select a job and click CANCEL to cancel it.</p>'),
    job_table_html, job_start_html,
    widgets.HBox([job_dropdown, cancel_button, refresh_button], layout=widgets.Layout(margin='10px 0')),
    job_status_output
], layout=widgets.Layout(padding='10px'))

orca_left = widgets.VBox([
    orca_row1,
    orca_row2,
    orca_row3,
    orca_row4,
    orca_row5,
    orca_row6,
    orca_row_solv,
    orca_row_opts,
    orca_row7,
    orca_row8,
    orca_row9,
    orca_row_upload,
    orca_row_buttons,
    orca_output
], layout=widgets.Layout(width='45%', padding='10px'))

orca_right = widgets.VBox([
    widgets.HTML('<b>ORCA Input Preview (editable):</b>'),
    orca_preview,
    widgets.HTML('<br><b>Molecule Preview:</b>'),
    orca_mol_output
], layout=widgets.Layout(width='55%', padding='10px'))

tab4_content = widgets.VBox([
    widgets.HTML('<h3>ORCA Input Builder</h3>'),
    widgets.HBox([orca_left, orca_right])
], layout=widgets.Layout(padding='10px'))

# === Create Tabs ===
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content])
tabs.set_title(0, 'Submit Job')
tabs.set_title(1, 'Recalc')
tabs.set_title(2, 'Job Status')
tabs.set_title(3, 'ORCA Builder')

display(widgets.HTML('<h2 style="color:#1976d2;">DELFIN Dashboard</h2>'))
display(tabs)


HTML(value='<h2 style="color:#1976d2;">DELFIN Dashboard</h2>')